In [39]:
import geopandas as gpd
from sentinelhub import Geometry, CRS

aoi_gdf   = gpd.read_file(r"anomalous fields/anomalous_fields_polygons.geojson")          # GeoJSON in EPSG:4326
aoi_geom  = Geometry(aoi_gdf.unary_union, CRS.WGS84)        # merge to one multipolygon
field_gdf = aoi_gdf.assign(fid=aoi_gdf.index)               # keep per-field IDs


/tmp/ipykernel_38823/434952207.py:5: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  aoi_geom  = Geometry(aoi_gdf.unary_union, CRS.WGS84)        # merge to one multipolygon


In [40]:
import pandas as pd, datetime as dt
start, end = dt.date(2023, 1, 1), dt.date(2023, 12, 31)
weeks = pd.date_range(start, end, freq="7D").date


In [41]:
from sentinelhub import SentinelHubRequest, DataCollection, MosaickingOrder, bbox_to_dimensions, SHConfig
from shapely.geometry import mapping

config = SHConfig()
config.instance_id = "919df252-b78d-4f73-b3ea-388af21f9464"
config.sh_client_id = "881396a0-464e-49f8-aef7-1f91b07ad08b"
config.sh_client_secret = "biPULdIEQf6BfjkZ0oRXnwuiseCgsBui"
PIX_RES  = 10          # metres
width, height = bbox_to_dimensions(aoi_geom.bbox, resolution=PIX_RES)

client_responses_all_bands = []
for w0, w1 in zip(weeks[:-1], weeks[1:]):
    request = SentinelHubRequest(
        evalscript=open("eval_script_all_bands.js").read(),
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL2_L2A,
                time_interval=(w0.isoformat(), w1.isoformat()),
                mosaicking_order=MosaickingOrder.LEAST_CC,
                maxcc=0.2
            )
        ],
        responses=[
            SentinelHubRequest.output_response("default", response_format='tiff')
        ],
        geometry=aoi_geom,
        size=(width, height),
        config=config
    )
    client_responses_all_bands.append(request.get_data()[0])  # ndarray (H,W,13)
print("Downloaded all bands for all weeks.")


Downloaded all bands for all weeks.


In [42]:
import numpy as np, rasterio, pandas as pd
from rasterio.transform import from_bounds

records = []
# Get bounding box coordinates
minx, miny, maxx, maxy = aoi_geom.bbox
# Use the same width and height as in the request
transform = from_bounds(minx, miny, maxx, maxy, width, height)

for i, arr in enumerate(client_responses, start=1):
    ndvi, evi, mask = arr[...,0], arr[...,1], arr[...,2]
    rows, cols = np.where(mask == 1)
    xs, ys = rasterio.transform.xy(transform, rows, cols, offset='center')
    for x, y, n, e in zip(xs, ys, ndvi[rows, cols], evi[rows, cols]):
        records.append(
            dict(lon=x, lat=y, ndvi=n, evi=e,
                 week=i, cloud=0,         # 0 = clear
                 id=None)                 # fill later via spatial join
        )

df = pd.DataFrame(records)

In [43]:
df_points = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.lon, df.lat), crs="EPSG:4326"
)
df_joined = gpd.sjoin(df_points, field_gdf[["fid","geometry"]], how="left")
df_joined.rename(columns={"fid":"id"}, inplace=True)


In [44]:
expected = [
    "id","lon","lat","ndvi","evi",   # add more indices later
    "cloud","week"
]
df_final = df_joined[expected].round(6)
df_final.to_csv("banana_vi_timeseries.csv", index=False)


In [45]:
# Save all bands as GeoTIFFs
import rasterio
from rasterio.transform import from_bounds

minx, miny, maxx, maxy = aoi_geom.bbox
transform = from_bounds(minx, miny, maxx, maxy, width, height)
crs = "EPSG:4326"
band_names = [
    "B01","B02","B03","B04","B05","B06","B07","B08","B8A","B09","B11","B12","dataMask"
]

for i, arr in enumerate(client_responses_all_bands, start=1):
    out_path = f"sentinel2_allbands_week_{i}.tif"
    with rasterio.open(
        out_path,
        "w",
        driver="GTiff",
        height=arr.shape[0],
        width=arr.shape[1],
        count=arr.shape[2],
        dtype=arr.dtype,
        crs=crs,
        transform=transform,
    ) as dst:
        for band in range(arr.shape[2]):
            dst.write(arr[..., band], band + 1)
        dst.set_band_description(band + 1, band_names[band])
    print(f"Saved {out_path}")

Saved sentinel2_allbands_week_1.tif
Saved sentinel2_allbands_week_2.tif
Saved sentinel2_allbands_week_3.tif
Saved sentinel2_allbands_week_4.tif
Saved sentinel2_allbands_week_5.tif
Saved sentinel2_allbands_week_6.tif
Saved sentinel2_allbands_week_7.tif
Saved sentinel2_allbands_week_8.tif
Saved sentinel2_allbands_week_9.tif
Saved sentinel2_allbands_week_10.tif
Saved sentinel2_allbands_week_11.tif
Saved sentinel2_allbands_week_12.tif
Saved sentinel2_allbands_week_13.tif
Saved sentinel2_allbands_week_14.tif
Saved sentinel2_allbands_week_15.tif
Saved sentinel2_allbands_week_16.tif
Saved sentinel2_allbands_week_17.tif
Saved sentinel2_allbands_week_18.tif
Saved sentinel2_allbands_week_19.tif
Saved sentinel2_allbands_week_20.tif
Saved sentinel2_allbands_week_21.tif
Saved sentinel2_allbands_week_22.tif
Saved sentinel2_allbands_week_23.tif
Saved sentinel2_allbands_week_24.tif
Saved sentinel2_allbands_week_25.tif
Saved sentinel2_allbands_week_26.tif
Saved sentinel2_allbands_week_27.tif
Saved sent